# BART JD GENERATION WITH STOPWORDS

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  # Use only GPU device 0 (change the number as needed)

In [3]:
import pandas as pd
import pandas as pd
import re
from transformers import BartTokenizer, BartForConditionalGeneration, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from transformers import BartTokenizer, BartForConditionalGeneration, TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from transformers import BartTokenizer, BartForConditionalGeneration, TrainingArguments, Trainer


In [4]:
import re

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Replace special characters
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\t', ' ', text)
    
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Apply preprocessing to the resume_text and job_description columns
#df["resume_text"] = df["resume_text"].apply(preprocess_text)

In [5]:
model = BartForConditionalGeneration.from_pretrained("fine_tuned_bart_large")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")


In [6]:
def generate_job_description(resume_text, model, tokenizer):
    inputs = tokenizer.encode(resume_text + " <sep>", return_tensors="pt", padding="max_length", truncation=True)
    outputs = model.generate(inputs, max_length=512, num_return_sequences=1, no_repeat_ngram_size=3)
    return tokenizer.decode(outputs[0])


In [7]:
df_non_jd_all = pd.read_csv("df_all_no_jd.csv")

In [7]:
df["resume_text"] = df["resume_text"].apply(preprocess_text)

In [8]:
df

,Job_Designation,resume_text,JD,tokens
0,database administrator,system engineer system engineer system enginee...,NaN,801
1,database administrator,production sql server database administrator p...,NaN,890
2,"software developer,front end developer,web dev...",sql server database administrator sql server d...,NaN,504
3,"software developer,front end developer,web dev...",sr. technical sourcer sr. technical sourcer sr...,NaN,1123
4,"systems administrator,database administrator",senior sql database administrator senior sql d...,NaN,500
...,...,...,...,...
20551,"web developer,software developer",senior web developer & web project manager sen...,NaN,363
20552,systems administrator,project coordinator project coordinator servic...,NaN,466
20553,"web developer,software developer",sr. ui developer sr. ui developer sr. ui devel...,NaN,3158
20554,"web developer,software developer",network engineer network engineer network engi...,NaN,1637


In [53]:
df2= df[5000:6000]

## 

In [54]:
df2

,Job_Designation,resume_text,JD,tokens
5000,"web developer,software developer,front end dev...",eis senior it infrastructure project manger ei...,NaN,1185
5001,"project manager,systems administrator",third party risk analyst third party risk anal...,NaN,933
5002,project manager,front- end developer front-end developer front...,NaN,981
5003,project manager,associate experience designer associate experi...,NaN,434
5004,security analyst,senior it project manager senior it project ma...,NaN,1739
...,...,...,...,...
5995,"web developer,software developer,front end dev...",lead access coordinator lead access coordinato...,NaN,399
5996,project manager,front end developer front end developer front ...,NaN,446
5997,project manager,enterprise technology project associate enterp...,NaN,842
5998,security analyst,assistant professor assistant professor smyrna...,NaN,1709


In [8]:
def generate_job_descriptions(resume_texts, model, tokenizer):
    job_descriptions = []
    for resume_text in resume_texts:
        jd = generate_job_description(resume_text, model, tokenizer)
        job_descriptions.append(jd)
    return job_descriptions


In [56]:
import torch

if torch.cuda.is_available():
    device = 'cuda'
    print("Using GPU")
else:
    device = 'cpu'
    print("Using CPU")

model.to(device)

def generate_job_descriptions(resume_texts, model, tokenizer):
    inputs = tokenizer.batch_encode_plus(resume_texts, return_tensors="pt", padding="max_length", truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(**inputs, max_length=512, num_return_sequences=1, no_repeat_ngram_size=3)
    return [tokenizer.decode(output) for output in outputs]

BATCH_SIZE = 16

current_time = datetime.datetime.now()
time_str = current_time.strftime("%H:%M:%S")
print("Current time:", time_str)

all_generated_job_descriptions = []

for i in range(0, len(df2), BATCH_SIZE):
    resume_texts = df2["resume_text"][i:i + BATCH_SIZE].tolist()
    generated_job_descriptions = generate_job_descriptions(resume_texts, model, tokenizer)
    assert len(generated_job_descriptions) == len(resume_texts)
    all_generated_job_descriptions.extend(generated_job_descriptions)
    print(f'{i + BATCH_SIZE} done')

df2['JD'] = all_generated_job_descriptions

current_time = datetime.datetime.now()
time_str = current_time.strftime("%H:%M:%S")
print("Current time:", time_str)


Using GPU
Current time: 17:06:50


/tmp/ipykernel_406791/2106897937.py:27: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  resume_texts = df2["resume_text"][i:i + BATCH_SIZE].tolist()


16 done
32 done
48 done
64 done
80 done
96 done
112 done
128 done
144 done
160 done
176 done
192 done
208 done
224 done
240 done
256 done
272 done
288 done
304 done
320 done
336 done
352 done
368 done
384 done
400 done
416 done
432 done
448 done
464 done
480 done
496 done
512 done
528 done
544 done
560 done
576 done
592 done
608 done
624 done
640 done
656 done
672 done
688 done
704 done
720 done
736 done
752 done
768 done
784 done
800 done
816 done
832 done
848 done
864 done
880 done
896 done
912 done
928 done
944 done
960 done
976 done
992 done
1008 done
Current time: 17:53:12


/tmp/ipykernel_406791/2106897937.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['JD'] = all_generated_job_descriptions


In [13]:
import datetime

In [58]:
df2.to_csv("1000jdgen5000-6000.csv")

In [15]:
import torch

if torch.cuda.is_available():
    device = 'cuda'
    print("Using GPU")
    torch.cuda.empty_cache()
else:
    device = 'cpu'
    print("Using CPU")

model.to(device)

def generate_job_descriptions(resume_texts, model, tokenizer):
    inputs = tokenizer.batch_encode_plus(resume_texts, return_tensors="pt", padding="max_length", truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(**inputs, max_length=512, num_return_sequences=1, no_repeat_ngram_size=3)
    return [tokenizer.decode(output) for output in outputs]

BATCH_SIZE = 16

current_time = datetime.datetime.now()
time_str = current_time.strftime("%H:%M:%S")
print("Current time:", time_str)

all_generated_job_descriptions = []

for i in range(0, len(df2), BATCH_SIZE):
    resume_texts = df2["resume_text"][i:i + BATCH_SIZE].tolist()
    generated_job_descriptions = generate_job_descriptions(resume_texts, model, tokenizer)
    assert len(generated_job_descriptions) == len(resume_texts)
    all_generated_job_descriptions.extend(generated_job_descriptions)
    print(f'{i + BATCH_SIZE} done')

df2['JD'] = all_generated_job_descriptions

current_time = datetime.datetime.now()
time_str = current_time.strftime("%H:%M:%S")
print("Current time:", time_str)


Using GPU
Current time: 13:01:56


/tmp/ipykernel_548542/3809448456.py:28: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  resume_texts = df2["resume_text"][i:i + BATCH_SIZE].tolist()


16 done
32 done
48 done
64 done
80 done
96 done
112 done
128 done
144 done
160 done
176 done
192 done
208 done
224 done
240 done
256 done
272 done
288 done
304 done
320 done
336 done
352 done
368 done
384 done
400 done
416 done
432 done
448 done
464 done
480 done
496 done
512 done
528 done
544 done
560 done
576 done
592 done
608 done
624 done
640 done
656 done
672 done
688 done
704 done
720 done
736 done
752 done
768 done
784 done
800 done
816 done
832 done
848 done
864 done
880 done
896 done
912 done
928 done
944 done
960 done
976 done
992 done
1008 done
1024 done
1040 done
1056 done
Current time: 15:21:30


/tmp/ipykernel_548542/3809448456.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['JD'] = all_generated_job_descriptions


In [17]:

# Create new DataFrame with 'JD' column initialized to all_generated_job_descriptions
new_df = pd.DataFrame({'JD': all_generated_job_descriptions})



In [18]:
new_df


,JD
0,</s><s>job title: it consultant (contract) loc...
1,</s><s><s><s>job title: senior infrastructure ...
2,</s><s><s><s>job title: sap basis administrato...
3,</s><s><s><s>job title: systems administrator/...
4,</s><s><s><s>job title: php web developer job ...
...,...
403,</s><s>job title: human factors specialist loc...
404,</s><s><s><s>job title: office administrator/f...
405,</s><s><s><s>job title: unix/linux systems adm...
406,</s><s>job title: systems engineer job summary...


In [79]:
new_df
# Concatenate new_df with df2 using axis=1 to add 'JD' column to the beginning of df2
df2 = pd.concat([new_df, df2], axis=1)

In [21]:
df2['JD'] = all_generated_job_descriptions


/tmp/ipykernel_529294/2761543922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['JD'] = all_generated_job_descriptions


In [16]:
df2

,Job_Designation,resume_text,JD,tokens
19504,"web developer,software developer",senior ux|ui designer senior ux|ui designer se...,</s><s>job title: senior ux/ui designer locati...,330
19505,"web developer,software developer",senior exchange engineer & systems administrat...,</s><s><s><s>job title: senior exchange engine...,1453
19506,"web developer,software developer,java developer",systems administrator/ devops engineer systems...,</s><s>job title: systems administrator/devops...,1501
19507,"software developer,web developer,systems admin...",desktop support ii desktop support ii desktop ...,</s><s><s><s>job title: desktop support ii loc...,492
19508,systems administrator,customer engineer customer engineer customer e...,</s><s>job title: customer engineer location: ...,556
...,...,...,...,...
20551,"web developer,software developer",senior web developer & web project manager sen...,</s><s><s><s>job title: senior web developer &...,363
20552,systems administrator,project coordinator project coordinator servic...,</s><s>job title: project coordinator/service ...,466
20553,"web developer,software developer",sr. ui developer sr. ui developer sr. ui devel...,</s><s><s><s>job title: senior ui developer jo...,3158
20554,"web developer,software developer",network engineer network engineer network engi...,</s><s>job title: network engineer location: t...,1637


In [17]:
df2.isnull().sum()

Job_Designation    0
resume_text        0
JD                 0
tokens             0
dtype: int64

In [18]:
df2.to_csv("1052jdgen19504:20556.csv")